# TP OpenMP - October 2022
## Part 1 : Pi
### Compilation

In [43]:
!g++ -o tp_openmp_part_1_pi_impl_critical part1/tp_openmp_part_1_pi_impl_critical.cpp -fopenmp -O3 -march=native
!g++ -o tp_openmp_part_1_pi_impl_reduce part1/tp_openmp_part_1_pi_impl_reduce.cpp -fopenmp -O3 -march=native
!g++ -o tp_openmp_part_1_pi_impl_atomic part1/tp_openmp_part_1_pi_impl_atomic.cpp -fopenmp -O3 -march=native

### Performance evaluation

In [44]:
import os
import subprocess
import pandas as pd

try:
    os.remove("stats.csv")
except OSError:
    pass

import pandas as pd

df = pd.DataFrame(columns=['version','nbcore','num_steps','runtime'])
df.to_csv("stats.csv", index=False)

num_steps = [10, 100, 1000]#, 1000000000000]
nb_threads = [1, 2, 4, 8]
repeats = range(0,10)

for nsteps in num_steps:
    for nthread in nb_threads:
        for repeat in repeats:
            args = ("./tp_openmp_part_1_pi_impl_critical", "-T", str(nthread), "-N", str(nsteps))
            popen = subprocess.Popen(args, stdout=subprocess.PIPE)
            popen.wait()

            
            args = ("./tp_openmp_part_1_pi_impl_reduce", "-T", str(nthread), "-N", str(nsteps))
            popen = subprocess.Popen(args, stdout=subprocess.PIPE)
            popen.wait()

            args = ("./tp_openmp_part_1_pi_impl_atomic", "-T", str(nthread), "-N", str(nsteps))
            popen = subprocess.Popen(args, stdout=subprocess.PIPE)
            popen.wait()
            



### Performance analysis
#### Weak scaling

In [45]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import genfromtxt

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('stats.csv',header=0,names=['version','nbcore','num_steps','runtime'],dtype={
                     'version': str,
                     'nbcore': int,
                     'num_steps' : int,
                     'runtime' : float
                 })
print(df)

color_num_steps = {10 : "blue", 100 : "red", 1000 : "green", 1000000000000 : "black"}

for num_steps in df['num_steps']:
    df_plot = df[(df['num_steps'] == int(num_steps))]
    df_plot = df_plot[df_plot['version'] == "atomic"]
    
    mean_stats = df_plot.groupby(['num_steps','version','nbcore']).mean().reset_index()
    
    plt.plot(mean_stats['nbcore'], mean_stats['runtime'],linestyle="solid",color=color_num_steps[num_steps])
    plt.yscale('log')
    plt.xscale('log')
    plt.scatter(df_plot['nbcore'], df_plot['runtime'],color=color_num_steps[num_steps])

    df_plot = df[(df['num_steps'] == num_steps) & (df['version'] == "reduce")]
    mean_stats = df_plot.groupby(['num_steps','version','nbcore']).mean().reset_index()
    
    plt.plot(mean_stats['nbcore'], mean_stats['runtime'],linestyle="dashed",color=color_num_steps[num_steps])
    plt.yscale('log')
    plt.xscale('log')
    plt.scatter(df_plot['nbcore'], df_plot['runtime'],color=color_num_steps[num_steps])
    
plt.legend()
plt.show()

      version  nbcore  num_steps   runtime
0    critical       1         10  0.000004
1      reduce       1         10  0.000004
2      atomic       1         10  0.000004
3    critical       1         10  0.000004
4      reduce       1         10  0.000004
5      atomic       1         10  0.000004
6    critical       1         10  0.000006
7      reduce       1         10  0.000004
8      atomic       1         10  0.000004
9    critical       1         10  0.000004
10     reduce       1         10  0.000003
11     atomic       1         10  0.000005
12   critical       1         10  0.000004
13     reduce       1         10  0.000003
14     atomic       1         10  0.000004
15   critical       1         10  0.000004
16     reduce       1         10  0.000004
17     atomic       1         10  0.000005
18   critical       1         10  0.000005
19     reduce       1         10  0.000005
20     atomic       1         10  0.000006
21   critical       1         10  0.000006
22     redu

KeyError: 10

#### Strong Scaling

### Discussion